In [1]:
import torch
import torchaudio
from tqdm import tqdm

In [2]:
class CFG:
    device='cuda'
    RATE=16000
    save_folder='slices'
    thresold1=0.5
    thresold=0.5
    batch_size=8
    epoches=1000
    fuck=999
    lr=3e-5
    strides=10

In [3]:
datas=[]
with open('list.txt','r',encoding='utf-8') as f:
    rows=f.readlines()
    for i,row in enumerate(rows):
        if(row==''):
            break
        id,setence=row.split(':')
        # print(id)
        id=int(id)
        datas.append({'id':id,'sentence':setence,'wav':CFG.save_folder+'/{}.wav'.format(id)})

In [4]:
def Load_Wav(filename = "./slices/1.wav"):
    waveform,sample_rate = torchaudio.load(filename)
    if(len(waveform[0])<10):
        return waveform[0].cpu().numpy()
    # print("Shape of waveform:{}".format(waveform.size())) #音频大小
    # print("sample rate of waveform:{}".format(sample_rate))#采样率
    resampler = torchaudio.transforms.Resample(sample_rate, CFG.RATE)
    if(sample_rate!=CFG.RATE):
        waveform=resampler(waveform)
    return waveform[0].cpu().numpy()

## WaveLM

In [5]:
from transformers import Wav2Vec2FeatureExtractor, WavLMForXVector
pretrained = WavLMForXVector.from_pretrained("microsoft/wavlm-base-plus-sv")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("microsoft/wavlm-base-plus-sv")
pretrained=pretrained.to(CFG.device)

In [24]:
import math
import pickle

tmp=[]
for i,data in enumerate(datas):
    try:
        tmp.append(Load_Wav(data['wav']))
    except:
        print(i)
        break

x=feature_extractor(tmp, return_tensors="pt",sampling_rate=CFG.RATE, padding=True).to(CFG.device)
# print(x)
# 'input_values','attention_mask'
# exit()


n=len(x['input_values'])
strides=int(math.ceil(n/CFG.batch_size))
embeddings=[]
with torch.no_grad():
    for stride in tqdm(range(strides)):
        st=stride*CFG.batch_size
        ed=min(st+CFG.batch_size,n)
        x_={'input_values':x['input_values'][st:ed],'attention_mask':x['attention_mask'][st:ed]}
        embeddings.append(pretrained(**x_).embeddings.detach())

embeddings=torch.cat(embeddings,dim=0)
with open('embeddings.pkl','wb') as f:
    pickle.dump(embeddings,f,-1)

## Downstream model

In [6]:
from torch import nn
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.fc1=nn.Linear(512,32)
        self.fc2=nn.Linear(32,2)
        self.softmax=nn.Softmax()

    def forward(self,x):
        x=x.to(CFG.device)
        x=self.fc1(x)
        x=self.fc2(x)
        x=self.softmax(x)
        return x

model=Model().to(CFG.device)

### Artificial classify

In [7]:
import torch
import time
import tkinter as tk
import pygame
import _thread
import time

audio_file = "tmp.wav"
cls=-1

def play_audio():
    try:
        pygame.mixer.init()
        audio = pygame.mixer.Sound(audio_file)
        _thread.start_new_thread(audio.play,())
    except:
        pass
    # audio.play()

def classify_as_1():
    global cls
    cls=1

def classify_as_0():
    global cls
    cls=0

is_running=True

def on_closing(root):
    global is_running  # 标记主线程结束
    is_running = False
    root.destroy()

def init():
    global play_button
    # 创建 GUI 窗口
    root = tk.Tk()
    # 设置窗口标题和尺寸
    root.title("Wav Classification")
    root.wm_attributes("-topmost", True)
    root.geometry("400x200")
    play_button = tk.Button(root, text="Wait a moment...", command=play_audio)
    play_button.pack()

    # 创建两个按钮来决定音频的分类
    classify_1_button = tk.Button(root, text="Positive", command=classify_as_1)
    classify_1_button.pack(side=tk.LEFT, padx=50)

    classify_2_button = tk.Button(root, text="Negative", command=classify_as_0)
    classify_2_button.pack(side=tk.RIGHT, padx=50)
    root.protocol("WM_DELETE_WINDOW", lambda:on_closing(root))
    return root


def UI():
    root=init()
    root.mainloop()

def Test(wavepth):
    time.sleep(0.2)
    global cls
    global audio_file
    cls=-1
    audio_file=wavepth
    play_audio()
    while is_running:
        play_button.config(text='Play '+audio_file)
        # print(cls,end=' ')
        if(cls!=-1):
            if(cls==0):
                return torch.as_tensor([1.,0.])
            else:
                return torch.as_tensor([0.,1.])
        time.sleep(0.2)

pygame 2.1.2 (SDL 2.0.18, Python 3.8.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Prepare

In [8]:
import pickle
from transformers import AdamW
import math
import os

f=open('embeddings.pkl','rb')
embeddings=pickle.load(f).to(CFG.device)
print(embeddings.shape)

optimizer=AdamW(model.parameters(),lr=CFG.lr)

train=[]

def Train():
    for epoch in tqdm(range(CFG.epoches)):
        tmp_x=[]
        tmp_label=[]
        for data in train:
            embed,label=data
            tmp_x.append(embed)
            tmp_label.append(label)
        tmp_x=torch.stack(tmp_x)
        tmp_label=torch.stack(tmp_label).to(CFG.device)
        out=model(tmp_x)
        loss=((out-tmp_label)**2).mean()
        if(epoch%CFG.fuck==0):
            print("stride={0} epoch={1} loss={2}".format(stride,epoch,loss.item()))
        loss.backward()
        optimizer.step()

torch.Size([1201, 512])


c:\Users\Hasee\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### First train

In [9]:

folder0 = r'./samples/0'
folder1 = r'./samples/1'

tmp0=[]
tmp1=[]
for root, dirs, files in os.walk(folder0):
    # print
    for file in files:
        # print('fuckpps')
        path = os.path.join(root, file)
        wav=Load_Wav(path)
        tmp0.append(wav)

for root, dirs, files in os.walk(folder1):
    # print
    for file in files:
        # print('fuckpps')
        path = os.path.join(root, file)
        wav=Load_Wav(path)
        tmp1.append(wav)

x0=feature_extractor(tmp0, return_tensors="pt",sampling_rate=CFG.RATE, padding=True).to(CFG.device)
x1=feature_extractor(tmp1, return_tensors="pt",sampling_rate=CFG.RATE, padding=True).to(CFG.device)

n0=len(x0['input_values'])
strides0=int(math.ceil(n0/CFG.batch_size))
embeddings0=[]
with torch.no_grad():
    for stride in tqdm(range(strides0)):
        st=stride*CFG.batch_size
        ed=min(st+CFG.batch_size,n0)
        x_={'input_values':x0['input_values'][st:ed],'attention_mask':x0['attention_mask'][st:ed]}
        embeddings0.append(pretrained(**x_).embeddings.detach())
embeddings0=torch.cat(embeddings0,dim=0)

n1=len(x1['input_values'])
strides1=int(math.ceil(n1/CFG.batch_size))
embeddings1=[]
with torch.no_grad():
    for stride in tqdm(range(strides1)):
        st=stride*CFG.batch_size
        ed=min(st+CFG.batch_size,n1)
        x_={'input_values':x1['input_values'][st:ed],'attention_mask':x1['attention_mask'][st:ed]}
        embeddings1.append(pretrained(**x_).embeddings.detach())
embeddings1=torch.cat(embeddings1,dim=0)
        
label=torch.as_tensor([1.,0.])
for embed in embeddings0:
    train.append([embed,label])
label=1-label
for embed in embeddings1:
    train.append([embed,label])
Train()
# train.clear()


  0%|          | 0/2 [00:00<?, ?it/s]c:\Users\Hasee\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\models\wavlm\modeling_wavlm.py:1755: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
  0%|          | 0/1000 [00:00<?, ?it/s]C:\Users\Hasee\AppData\Local\Temp\ipykernel_6724\929340445.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.softmax(x)
  7%|▋         | 74/1000 [00:00<00:01, 740.00it/s]

stride=0 epoch=0 loss=0.26160338521003723


100%|██████████| 1000/1000 [00:01<00:00, 784.75it/s]

stride=0 epoch=999 loss=0.07544083148241043


### Train

In [10]:
_thread.start_new_thread(UI,())
apr=[]
for stride in range(CFG.strides):
    with torch.no_grad():
        out=model(embeddings)
        sum=0
        for i,p in enumerate(out):
            if(p[1]<CFG.thresold1):
                continue
            if(i in apr):
                continue
            label=Test(datas[i]['wav'])
            apr.append(i)
            # print('fuckpps')
            train.append([embeddings[i],label])
            if(label[1]>=1.):
                sum+=1
            if sum>=10:
                break
    Train()
    # train.clear()
is_running=False

C:\Users\Hasee\AppData\Local\Temp\ipykernel_6724\929340445.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.softmax(x)
  4%|▍         | 40/1000 [00:00<00:02, 393.39it/s]

stride=0 epoch=0 loss=0.23380403220653534


100%|██████████| 1000/1000 [00:01<00:00, 674.07it/s]


stride=0 epoch=999 loss=0.134284108877182


  9%|▊         | 86/1000 [00:00<00:02, 423.89it/s]

...

stride=8 epoch=999 loss=0.09944672137498856


  3%|▎         | 34/1000 [00:00<00:02, 330.27it/s]

stride=9 epoch=0 loss=0.10856372117996216


100%|██████████| 1000/1000 [00:01<00:00, 644.14it/s]

stride=9 epoch=999 loss=0.10530161112546921


## Save

In [11]:
out=model(embeddings)
result=[]
for j,p in enumerate(out):
    if(p[1]<CFG.thresold):
        continue
    result.append(j)

C:\Users\Hasee\AppData\Local\Temp\ipykernel_6724\929340445.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.softmax(x)


In [12]:
import pickle
print(len(result))
with open('result.pkl','wb') as f:
    pickle.dump(result,f,-1)

123


In [13]:
import os
def mkdir(pth):
    if(not os.path.exists(pth)):
        os.mkdir(pth)

In [15]:
import pickle
import shutil
f=open('result.pkl','rb')
result=pickle.load(f)
mkdir('result')
mkdir('result/audios')
# shutil.copy('list.txt','result/list_.txt')
f_list=open('result/result.txt','w',encoding='utf-8')
for i,id in enumerate(result):
    ele=datas[id]
    wav=ele['wav']
    text=ele['sentence']
    shutil.copy(wav,'result/audios/{0}.wav'.format(i))
    f_list.write(str(j)+':'+text)
f_list.close()